<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

Pulling some stats for a partner request from sistas in prison reform, no spark modeling here. Uses the data we scraped in October 2021

In [3]:
import os
import sys
sys.path.insert(0, os.path.relpath('../../../../..')) 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from recidiviz.calculator.modeling.population_projection.super_simulation.super_simulation_factory import SuperSimulationFactory
from recidiviz.calculator.modeling.population_projection.transition_table import TransitionTable
from recidiviz.calculator.modeling.population_projection.shell_compartment import ShellCompartment
from recidiviz.calculator.modeling.population_projection.spark_policy import SparkPolicy
from recidiviz.calculator.modeling.population_projection.utils.spark_bq_utils import upload_spark_model_inputs
import pandas as pd
from functools import partial

pd.set_option('display.max_rows', 500)
pd.set_option('max_columns', 100)

In [4]:
sdc_data = pd.concat([
    pd.read_csv(
        f'../state/VA/VA_data/VA_circuit_data/circuit_data_{i}.csv',
        index_col=0
    ) for i in range(8)
])

In [5]:
scale_factor = len(sdc_data) / len(sdc_data[sdc_data.crime_type.notnull()])
sorted_data = sdc_data[sdc_data.crime_type.notnull()]

LOS_BY_CRIME = {
    'CONSPIRACY': 0.899,
    'DRUG_POSSESSION': 0.897,
    'DRUG_SALE': 0.902,
    'DRUG_OTHER': 0.901,
    'KIDNAPPING': 0.913,
    'MANSLAUGHTER': 0.916,
    'LARCENY': 0.907,
    'WEAPON': 0.905,
    'ASSAULT': 0.912,
    'ROBBERY': 0.920,
    'BURGLARY': 0.917,
    'DWI': 0.893,
    'RAPE': 0.906,
    'SEX_OFFENSE': 0.90,
    'ARSON': 0.914,
    'FIRST_DEGREE_MURDER': 0.907,
    'SECOND_DEGREE_MURDER': 0.914,
    'OTHER_MURDER': 0.908,
    'CAPITAL_MURDER': 0.898
}

sorted_data['LOS'] = sorted_data.apply(lambda x: LOS_BY_CRIME[x.crime_type] * x.total_sentence_months, axis=1)
sorted_data['end_date'] = sorted_data.time_step + sorted_data.LOS.apply(round)

<ipython-input-5-045a1f98fc5c>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sorted_data['LOS'] = sorted_data.apply(lambda x: LOS_BY_CRIME[x.crime_type] * x.total_sentence_months, axis=1)
<ipython-input-5-045a1f98fc5c>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sorted_data['end_date'] = sorted_data.time_step + sorted_data.LOS.apply(round)


In [30]:
currently_incarcerated = sorted_data[(sorted_data.end_date > 138) & (sorted_data.time_step < 138)]
currently_incarcerated.LOS = currently_incarcerated.LOS.round()
len(currently_incarcerated) / len(sorted_data)

/Users/pacopoler/.local/share/virtualenvs/recidiviz-data-Jd9N3JcA/lib/python3.8/site-packages/pandas/core/generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


0.029778182450993798

In [25]:
weapon_incarcerated.head()

,Address,Amended Charge,Amended Charge Type,Amended Code Section,Arrest Date,Charge,Charge Type,Class,CodeSection,Commenced by,Concluded By,ConcurrentConsecutive,Costs,Court/DMV Surrender,DOB,Defendant,Disposition Date,Driver Improvement Clinic,Driving Restrictions,Filed,Fine Amount,Fines/Cost Paid,Jail/Penitentiary,Life/Death,Locality,Military,Offense Date,Operator License Suspension Time,Probation Starts,Probation Time,Probation Type,Program Type,Race,Restitution Amount,Restitution Paid,Sentence Suspended,Sentence Time,Sex,Traffic Fatality,VA Alcohol Safety Action,county,time_step,day,month,year,sentence_time,sentence_suspended,person_id,compartment_duration,CONSPIRACY,FIRST_DEGREE_MURDER,SECOND_DEGREE_MURDER,CAPITAL_MURDER,OTHER_MURDER,DRUG_POSSESSION,DRUG_SALE,DRUG_OTHER,ARSON,SEX_OFFENSE,DWI,WEAPON,BURGLARY,ROBBERY,LARCENY,ASSAULT,RAPE,KIDNAPPING,MANSLAUGHTER,crimes,CONSPIRACY_ANY,DRUG_POSSESSION_ANY,DRUG_SALE_ANY,DRUG_OTHER_ANY,KIDNAPPING_ANY,MANSLAUGHTER_ANY,LARCENY_ANY,WEAPON_ANY,ASSAULT_ANY,ROBBERY_ANY,BURGLARY_ANY,DWI_ANY,RAPE_ANY,SEX_OFFENSE_ANY,ARSON_ANY,FIRST_DEGREE_MURDER_ANY,SECOND_DEGREE_MURDER_ANY,OTHER_MURDER_ANY,CAPITAL_MURDER_ANY,total_sentence_months,crime_type,LOS,end_date,served_so_far
182,LYNCHBURG[.] VA 24503,NaN,NaN,NaN,05/05/2020,POSSESS GUN W/DRUGS,Felony,6,18.2-308.4,Indictment,Guilty Plea,Sentence Is Run Consecutively With Another,NaN,NaN,01/18/****,ABBETT[.] RAY ALLEN,03/16/2021,NaN,NaN,07/20/2020,NaN,NaN,Penitentiary,NaN,COMMONWEALTH OF VA,NaN,05/05/2020,NaN,Probation To Begin Upon Release,4 Year(s),Supervised,NaN,White,NaN,NaN,4 Year(s),5 Year(s),Male,NaN,NaN,Botetourt Circuit Court,135.526027,16,3,2021,5 Year(s),4 Year(s),ABBETT[.] RAY ALLEN01/18/****LYNCHBURG[.] VA 2...,12.0,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,4,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,12.0,WEAPON,10.86,146.526027,2.473973
1164,RICHMOND[.] VA 23223,NaN,NaN,NaN,05/12/2018,POSS FIREARM W/CONT SUB W/INT,Felony,6,18.2-308.4,Direct Indictment,Guilty Plea,NaN,$421.00,NaN,03/28/****,ABERNATHY[.] THOMAS DENZEL,08/13/2018,NaN,NaN,05/09/2018,$0.00,NaN,Penitentiary,NaN,COMMONWEALTH OF VA,NaN,09/20/2017,NaN,NaN,NaN,NaN,NaN,Black,$0.00,NaN,NaN,5 Year(s),Male,NaN,NaN,Richmond City Circuit Court,104.427397,13,8,2018,5 Year(s),0,ABERNATHY[.] THOMAS DENZEL03/28/****RICHMOND[....,60.0,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,3,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,60.0,WEAPON,54.30,158.427397,33.572603
2072,STAUNTON[.] VA 24401,NONVIOL FELON POSS GUN W/I 10Y,Felony,18.2-308.2,12/05/2019,VIOLENT FELON POSS/TRAN WEAPON,Felony,6,18.2-308.2,Indictment,Guilty Plea,Sentence Is Run Consecutively With Another,NaN,NaN,09/19/****,ACKELSON[.] DERICK HEATH,12/18/2020,NaN,NaN,03/25/2014,NaN,NaN,Penitentiary,NaN,COMMONWEALTH OF VA,NaN,12/05/2013,NaN,NaN,NaN,NaN,NaN,White,NaN,NaN,NaN,2 Year(s),Male,NaN,NaN,Waynesboro Circuit Court,132.591781,18,12,2020,2 Year(s),0,ACKELSON[.] DERICK HEATH09/19/****STAUNTON[.] ...,24.0,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,2,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,24.0,WEAPON,21.72,154.591781,5.408219
2073,STAUNTON[.] VA 24401,NONVIOL FELON POSS GUN W/I 10Y,Felony,18.2-308.2,12/13/2013,VIOLENT FELON POSS/TRAN WEAPON,Felony,6,18.2-308.2,Indictment,Guilty Plea,NaN,$443.00,NaN,09/19/****,ACKELSON[.] DERICK HEATH,04/29/2021,NaN,NaN,05/22/2014,$0.00,NaN,Penitentiary,NaN,COMMONWEALTH OF VA,NaN,09/13/2013,NaN,Probation To Begin Upon Release,2 Year(s),Supervised,NaN,White,$0.00,NaN,3 Year(s),5 Year(s),Male,No,NaN,Augusta Circuit Court,136.953425,29,4,2021,5 Year(s),3 Year(s),ACKELSON[.] DERICK HEATH09

In [12]:
ETC_LEVEL_BY_CRIME = pd.DataFrame({
    'CONSPIRACY': {1:27, 2: 5, 3: 12, 4:7},
    'DRUG_POSSESSION': {1:574, 2:78, 3:177, 4:171},
    'DRUG_SALE': {1:1415, 2:245, 3:629, 4:458},
    'DRUG_OTHER': {1:1989, 2:323, 3:806, 4:629},
    'KIDNAPPING': {1:641, 2:203, 3:430, 4:304},
    'MANSLAUGHTER': {1:175, 2:44, 3:122, 4:96},
    'LARCENY': {1:1289, 2:190, 3:447, 4:591},
    'WEAPON': {1:346, 2:45, 3:100, 4:158},
    'ASSAULT': {1:1391, 2:354, 3:685, 4:728},
    'ROBBERY': {1:1491, 2:488, 3:1049, 4:1014},
    'BURGLARY': {1:740, 2:165, 3:413, 4:476},
    'DWI': {1: 101, 2: 14, 3: 28, 4:24},
    'RAPE': {1:1355, 2:304, 3:917, 4:405},
    'SEX_OFFENSE': {1:343, 2:60, 3:180, 4:80},
    'ARSON': {1:40, 2:13, 3:14, 4:26},
    'FIRST_DEGREE_MURDER': {1:868, 2:288, 3:534, 4:313},
    'SECOND_DEGREE_MURDER': {1:296, 2: 109, 3:224, 4:148},
    'OTHER_MURDER': {1:1327, 2:432, 3:833, 4:498},
    'CAPITAL_MURDER': {1:163, 2:35, 3:75, 4:37}
}).transpose()

LEVEL_ONE = ETC_LEVEL_BY_CRIME[1] / ETC_LEVEL_BY_CRIME.sum(axis=1)

In [31]:
robbery_incarcerated = currently_incarcerated[currently_incarcerated.crime_type == 'ROBBERY']
len(robbery_incarcerated) / len(currently_incarcerated)

0.07534568223323768

In [32]:
robbery_incarcerated.groupby('Race').count().max(axis=1)

Race
Asian Or Pacific Islander                       1
Black                                         970
Hispanic                                        5
Other (Includes Not Applicable[.] Unknown)      8
White                                         459
dtype: int64

In [33]:
weapon_incarcerated = currently_incarcerated[currently_incarcerated.crime_type == 'WEAPON']
len(weapon_incarcerated) / len(currently_incarcerated)

0.09449517349334725

In [34]:
weapon_incarcerated.groupby('Race').count().max(axis=1)

Race
Asian Or Pacific Islander                        5
Black                                         1075
Hispanic                                         2
Other (Includes Not Applicable[.] Unknown)       2
Unknown                                          5
White                                          722
dtype: int64

In [35]:
robbery_w_wpn = robbery_incarcerated[robbery_incarcerated.Charge.apply(lambda x: any([i in x for i in [
    'WPN',
    'WEAPON',
    'GUN',
    'FIREARM'
]]))]

In [36]:
robbery_w_wpn.groupby('Race').count().max(axis=1)

Race
Black    23
White    20
dtype: int64

In [49]:
LOS_BY_RACE = sdc_data[sdc_data.crime_type == 'WEAPON'].groupby(['Race', 'total_sentence_months']).count().max(axis=1).unstack('Race')
avg_LOS = LOS_BY_RACE.apply(lambda x: (LOS_BY_RACE.index * x).sum() / x.sum())
avg_LOS

Race
American Indian Or Alaskan Native              5.222222
Asian Or Pacific Islander                     11.660066
Black                                         18.061566
Hispanic                                      13.502701
Other (Includes Not Applicable[.] Unknown)    10.155439
Unknown                                        7.427221
White                                         12.501676
dtype: float64

In [39]:
sentence_table = pd.DataFrame()
sentence_lengths = [1 * 12, 3 * 12, 5 * 12, 7 * 12, 10 * 12, 15 * 12, 20 * 12, 25 * 12, 30 * 12, 35 * 12, 40 * 12, 45 * 12, 50 * 12, 55 * 12, 60 * 12, 65 * 12, 70 * 12, 75 * 12, 80 * 12]

for i, start in enumerate(sentence_lengths):
    sentence_group = pd.DataFrame()
    
    if i == (len(sentence_lengths) - 1):
        sentence_group = currently_incarcerated[
            currently_incarcerated.LOS >= start
        ]
    else:
        end = sentence_lengths[i + 1]
        
        sentence_group = currently_incarcerated[
            (currently_incarcerated.LOS >= start) & (currently_incarcerated.LOS < end)
        ]
    
    sentence_table[start/12] = sentence_group.groupby('crime_type').count().max(axis=1)
    
sentence_table.loc['total'] = time_served_table.sum()
sentence_table = time_served_table.fillna(0)
print(len(currently_incarcerated))
sentence_table

19165


,1.0,3.0,5.0,7.0,10.0,15.0,20.0,25.0,30.0,35.0,40.0,45.0,50.0,55.0,60.0,65.0,70.0,75.0,80.0
crime_type,,,,,,,,,,,,,,,,,,,
ARSON,32.0,10.0,5.0,7.0,5.0,2.0,1.0,4.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ASSAULT,812.0,333.0,117.0,136.0,149.0,65.0,30.0,16.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BURGLARY,291.0,124.0,58.0,63.0,55.0,40.0,14.0,3.0,7.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CAPITAL_MURDER,22.0,18.0,16.0,16.0,38.0,32.0,22.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CONSPIRACY,63.0,31.0,12.0,8.0,7.0,9.0,5.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DRUG_OTHER,474.0,131.0,47.0,15.0,5.0,4.0,3.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DRUG_POSSESSION,565.0,186.0,56.0,31.0,33.0,10.0,2.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DRUG_SALE,601.0,210.0,96.0,50.0,24.0,7.0,7.0,6.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DWI,49.0,16.0,4.0,6.0,0.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
actual_population_july_2021 = 24467
error = (len(currently_incarcerated) * scale_factor - actual_population_july_2021) / actual_population_july_2021
error

In [ ]:
LEVEL_ONE

In [ ]:
currently_incarcerated.LOS.median() / 12